In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns



train_data = pd.read_csv("train.csv")
train_data = train_data.drop('Id', axis=1)
print(train_data.columns.tolist())


# print(data['SalePrice'].describe())
# plt.figure(figsize=(9, 8))
# sns.distplot(data['SalePrice'], color='g', bins=100, hist_kws={'alpha': 0.4});

X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']

for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].fillna('None').astype('category')

for col in X.select_dtypes(include=['int64', 'float64']).columns:
    X[col] = X[col].fillna(X[col].median())

data_dmatrix = xgb.DMatrix(data=X, label=y, enable_categorical=True)

['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fen

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold

xgb_reg = XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    enable_categorical=True
)
parameter_grid = {
    'n_estimators': [800, 1200, 1600, 2000],
    'max_depth': [3, 6],
    'learning_rate': [0.01, 0.005],
    'min_child_weight': [1, 10, 100]
    }


cv_strategy = KFold(n_splits=5, shuffle=True, random_state=42)

random_search = RandomizedSearchCV(
    estimator=xgb_reg,
    param_distributions=parameter_grid,
    n_iter=25,
    scoring='neg_root_mean_squared_error',
    cv=cv_strategy,
    verbose=2,
    n_jobs=-1
)

random_search.fit(X, y)
print(random_search.best_params_)
model = random_search.best_estimator_


Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] END gamma=0.1, learning_rate=0.01, max_depth=3, min_child_weight=100, n_estimators=1200; total time=   0.8s
[CV] END gamma=0.1, learning_rate=0.01, max_depth=3, min_child_weight=100, n_estimators=1200; total time=   0.8s
[CV] END gamma=0.1, learning_rate=0.01, max_depth=3, min_child_weight=100, n_estimators=1200; total time=   0.8s
[CV] END gamma=0.1, learning_rate=0.01, max_depth=3, min_child_weight=100, n_estimators=1200; total time=   0.9s
[CV] END gamma=0.1, learning_rate=0.01, max_depth=3, min_child_weight=100, n_estimators=1200; total time=   0.9s
[CV] END gamma=0.2, learning_rate=0.005, max_depth=6, min_child_weight=100, n_estimators=1600; total time=   1.6s
[CV] END gamma=0.2, learning_rate=0.005, max_depth=6, min_child_weight=100, n_estimators=1600; total time=   1.5s
[CV] END gamma=0.2, learning_rate=0.005, max_depth=6, min_child_weight=100, n_estimators=1600; total time=   1.7s
[CV] END gamma=0.2, learning_ra

In [ ]:
# y_pred_train = model.predict(X)
# train_rmse = np.sqrt(mean_squared_error(np.log1p(y), np.log1p(y_pred_train)))
# print("Log-RMSE on training data:", train_rmse)

Log-RMSE on training data: 0.05205835529625534


In [ ]:
test_data = pd.read_csv("test.csv")
test_data = test_data.drop('Id', axis=1)
X_test = test_data.copy()


for col in X.select_dtypes(include='category').columns:
    if col in X_test.columns:
        X_test[col] = X_test[col].fillna('None').astype('category')

for col in X.select_dtypes(include=['int64','float64']).columns:
    if col in X_test.columns:
        X_test[col] = X_test[col].fillna(X[col].median())

#
y_pred = model.predict(X_test)  # use tuned model


In [33]:
# * Checking for overfitting

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.02,
random_state=42)

model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=50
)
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)

train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))

print("Training RMSE:", train_rmse)
print("Validation RMSE:", val_rmse)

[0]	validation_0-rmse:62204.54113
[1]	validation_0-rmse:61660.75635
[2]	validation_0-rmse:61212.58385
[3]	validation_0-rmse:60684.09582
[4]	validation_0-rmse:60246.21938
[5]	validation_0-rmse:59783.18050
[6]	validation_0-rmse:59302.10517
[7]	validation_0-rmse:58849.46225
[8]	validation_0-rmse:58379.68064
[9]	validation_0-rmse:57932.79947
[10]	validation_0-rmse:57518.20575
[11]	validation_0-rmse:57068.20630
[12]	validation_0-rmse:56631.09231
[13]	validation_0-rmse:56191.87782
[14]	validation_0-rmse:55766.77373
[15]	validation_0-rmse:55338.24567
[16]	validation_0-rmse:54959.48504
[17]	validation_0-rmse:54578.61621
[18]	validation_0-rmse:54163.35259
[19]	validation_0-rmse:53794.70972
[20]	validation_0-rmse:53434.64898
[21]	validation_0-rmse:53064.81971
[22]	validation_0-rmse:52710.41142
[23]	validation_0-rmse:52392.29008
[24]	validation_0-rmse:52036.46671
[25]	validation_0-rmse:51666.94444
[26]	validation_0-rmse:51327.79152
[27]	validation_0-rmse:51039.03982
[28]	validation_0-rmse:50714.1

/Users/connor/miniconda3/envs/ml/lib/python3.10/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[124]	validation_0-rmse:32021.90036
[125]	validation_0-rmse:31825.33981
[126]	validation_0-rmse:31709.26578
[127]	validation_0-rmse:31544.08974
[128]	validation_0-rmse:31442.24295
[129]	validation_0-rmse:31260.93883
[130]	validation_0-rmse:31162.77866
[131]	validation_0-rmse:31040.17053
[132]	validation_0-rmse:30868.29609
[133]	validation_0-rmse:30797.83268
[134]	validation_0-rmse:30679.70717
[135]	validation_0-rmse:30546.60529
[136]	validation_0-rmse:30455.92564
[137]	validation_0-rmse:30319.51649
[138]	validation_0-rmse:30177.69233
[139]	validation_0-rmse:30044.98630
[140]	validation_0-rmse:29976.40550
[141]	validation_0-rmse:29868.05373
[142]	validation_0-rmse:29718.05620
[143]	validation_0-rmse:29632.08404
[144]	validation_0-rmse:29506.55205
[145]	validation_0-rmse:29450.82771
[146]	validation_0-rmse:29337.90618
[147]	validation_0-rmse:29204.55410
[148]	validation_0-rmse:29146.29380
[149]	validation_0-rmse:29014.76032
[150]	validation_0-rmse:28897.98957
[151]	validation_0-rmse:2885